In [ ]:
import numpy as np
import cv2



mosaic_rate = 30
detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


video = cv2.VideoCapture(0)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
writer = cv2.VideoWriter('face_mosaic.avi', fourcc, 10.0, (640, 480))

i=0
ok=0
no=0
while (True):
    ret, img = video.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        roi=img[y:y+h, x:x+w]
        dst=cv2.resize(roi, dsize=(128,128),interpolation=cv2.INTER_AREA)
        dst_1=dst.reshape(1,49152)
        
        predict=sess.run(tf.argmax(logits, 1), feed_dict={X:dst_1, keep_prob : 1})[0]
        
        if predict == 0 :
            no=no+1
            print('no:',no)

            face_img = cv2.resize(roi, (w//mosaic_rate, h//mosaic_rate))
        # 확대/축소한 그림을 원래 크기로 돌리기
            face_img = cv2.resize(face_img, (w, h), interpolation=cv2.INTER_AREA)
        # 원래 이미지에 붙이기
            img[y:y+h, x:x+w] = face_img
        
        else :
            ok=ok+1
            print('ok:', ok)
        

    cv2.imshow('frame', img)
    writer.write(img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

        
video.release()
writer.release()
cv2.destroyAllWindows()